In [24]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from src.utils import load_data, load_config, load_merged_features
from src import utils_stats as us


from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from interpret import show

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
config = load_config("config.yaml")

data = load_merged_features(config)
data = data.replace(["MANQUANT"], 0)
data = data.replace([np.inf, -np.inf], 0)
#data = data.dropna()
data['ABS_PRE_DISSOCIATION'] = data['ABS_PRE_DISSOCIATION'].apply(int)
data['source_enc'] = data['source'].apply(lambda x : 1 if x == "quebec" else 0)


In [50]:
data.ABS_PRE_DISSOCIATION.value_counts()

ABS_PRE_DISSOCIATION
1    31
0    27
Name: count, dtype: int64

In [49]:
data.groupby("TYPE_TRAUMA").ABS_PRE_DISSOCIATION.value_counts()

TYPE_TRAUMA  ABS_PRE_DISSOCIATION
1            0                       16
             1                       13
2            1                       18
             0                       11
Name: count, dtype: int64

In [7]:
data.filter(regex = "inco").columns



Index(['Q1_incomplete_sentence', 'Q1_incomplete_word',
       'Q2_incomplete_sentence', 'Q2_incomplete_word',
       'Q3_incomplete_sentence', 'Q3_incomplete_word',
       'Q4_incomplete_sentence', 'Q4_incomplete_word',
       'text_incomplete_sentence', 'text_incomplete_word'],
      dtype='object')

In [36]:
X = data[['Q1_passive_percentages',
          "Q1_ON_NOUS","Q1_ON_QUELQU_UN","Q1_ON_GENERIQUE",
          "Q1_SENSATIONS_PHYSIQUES",
          "Q1_MORT_IMPLICITE",'Q1_MORT_EXPLICITE',"Q1_CORPS",
          'Q1_PRESENT_ENNONCIATION',  'Q1_PRESENT_GENERIQUE', 'Q1_PRESENT_HISTORIQUE',
          "Q1_silent_break","Q1_incomplete_sentence","Q1_filled_break","Q1_incomplete_word",
          "TRAITEMENT", "source_enc"
          ]]


In [42]:
y = data['ABS_PRE_DISSOCIATION']
seed = 5

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=seed)
ebm = LogisticRegression()
ebm.fit(X_train, y_train)

y_pred = ebm.predict(X_test)
print("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred) * 100.0))
print("F1: %.2f%%" % (f1_score(y_test, y_pred) * 100.0))

show(ebm.explain_global())

Accuracy: 22.22%
F1: 22.22%


<!-- http://127.0.0.1:7001/140270654044528/ -->

<!-- http://127.0.0.1:7001/140270678535136/ -->